# Run prerequisites

In [ ]:
import networkx as nx
import numpy as np
import random
import time

In [ ]:
def assign_weights_randomly(G):
  H = G.copy()
  for n in H.nodes:
    H.nodes[n]['node weight'] = 100 * (1 - random.uniform(0, 1))
  for e in H.edges:
    H.edges[e]['edge weight'] = 100 * (1 - random.uniform(0, 1))
  return H

def test_heuristics(G, heuristics, timeout=300, name="No name provided", c=0.2):
  results = []
  print("Graph:", name)
  print("n:", G.number_of_nodes(), "m:", G.number_of_edges())
  print("")
  for heuristic in heuristics:
    name = heuristic[0]
    h = heuristic[1]
    print("Testing", name)
    results.append((name, alg(h, G, c, timeout=timeout)))
    print("")

  return results

def test_graphs(graphs, g_type, heuristics, timeout=300, c=0.2):
  graph_results = []
  print("--- Running tests for", g_type, "graphs ---")
  print("")
  for graph in graphs:
    name = graph[0]
    G = graph[1]
    graph_results.append(test_heuristics(G, heuristics, timeout, name, c))
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("")
  
  print("")
  print("--- All tests for", g_type, "graphs size have now finished ---")
  return graph_results

def draw_results(results):
  for result in results:
    name = result[0]
    S, graph = result[1]
    print("Results for", name)
    print("Set S of removed vertices:", S)
    print("Resulting graph, G':")
    nx.draw(graph)

In [ ]:
def alg(heuristic, _G, c=3, timeout=300, name="Name not provided"):
  start = time.time()

  G = _G.copy()
  N = G.number_of_nodes()
  S = []

  # The connected components are ordered by decreasing size.
  gcc = max(nx.connected_components(G), key=len)
  size_gcc = len(list(gcc))

  robustness = 0

  while size_gcc > c:
    end = time.time()
    if round(time.time() - start, 3) > timeout:
      # Fail dismantling
      print("TIMEOUT -- Heuristic", name, "took longer than", timeout, "seconds")
      print("Here are the partial results:")
      cost = sum(_G.nodes[i]['node weight'] for i in S)
      if len(S) == 0:
        print("Removed no nodes from G.")
        print("Robustness is undefined")
      else: 
        robustness_score = robustness / _G.number_of_nodes()
        print(f"Removed {len(S)} nodes from G.")
        print("Robustness score:", robustness_score)
      print(f"Cost: {cost}")
      return S, G

    node_to_remove = heuristic(gcc, G)
    G.remove_node(node_to_remove)
    S.append(node_to_remove)

    components = nx.connected_components(G)
    gcc = max(nx.connected_components(G), key=len)
    size_gcc = len(list(gcc))
    robustness += size_gcc * _G.nodes[node_to_remove]['node weight']

  end = time.time()

  cost = sum(_G.nodes[i]['node weight'] for i in S)

  print(f"Removed {len(S)} nodes from G.")
  print(f"Cost: {cost}")
  if len(S) == 0:
    print("Removed no nodes from G.")
    print("Robustness is undefined")
  else: 
    robustness_score = robustness / _G.number_of_nodes()
    print(f"Removed {len(S)} nodes from G.")
    print("Robustness score:", robustness_score)
  
  print(f"Time elapsed: {round(end - start, 3)}s")

  return S, G


In [ ]:
# Generate a sea-urchin-sistine-chapel (SUSC) graph
def susc_generator(num_spikes, path_length, p=0):
  
  assert p <= 1
  assert p >= 0

  G = nx.Graph()
  k = 0 # constant to differentiate the two urchins
  
  # Urchin 1
  urchin_1 = 0
  G.add_node(urchin_1)
  k += 1

  for i in range(num_spikes):
    G.add_edge(k, urchin_1) # Add spike to urchin
    k += 1

  # Probabilistically add edges between spikes

  if p > 0:
    for i in range(num_spikes):
      for j in range(num_spikes):
        if i == j:
          continue
        spike_1 = i + 1 + urchin_1
        spike_2 = j + 1 + urchin_1
        if random.uniform(0, 1) <= p:
          G.add_edge(spike_1, spike_2)

  urchin_2 = k
  
  G.add_node(urchin_2)

  k += 1

  # Urchin 2


  for i in range(num_spikes):
    G.add_edge(k, urchin_2) # Add spike to urchin
    k += 1

  if p > 0:
    for i in range(num_spikes):
      for j in range(num_spikes):
        if i == j:
          continue
        spike_1 = i + urchin_2 + 1
        spike_2 = j + urchin_2 + 1
        if random.uniform(0, 1) <= p:
          G.add_edge(spike_1, spike_2)


  # Path

  if path_length == 0:
    G.add_edge(urchin_1, urchin_2)
  else:
    count = G.number_of_nodes() + 1
    start = count

    for i in range(path_length - 1):
      G.add_edge(count, count + 1)
      count += 1
    
    end = count

    G.add_edge(start, urchin_1)
    G.add_edge(end, urchin_2)

  return G

In [ ]:
### Generate graphs ###

## Small  ~ 100 nodes
## Medium ~ 500 nodes
## Large  ~ 1000 nodes
# Note that the numbers are not exactly that

# 1. BA graphs

BA_s = nx.barabasi_albert_graph(100, 10)
BA_s_weighted = assign_weights_randomly(BA_s)

BA_m = nx.barabasi_albert_graph(500, 50)
BA_m_weighted = assign_weights_randomly(BA_m)

BA_l = nx.barabasi_albert_graph(1000, 100)
BA_l_weighted = assign_weights_randomly(BA_l)

# 2. Regular graphs

Reg_s = nx.random_regular_graph(10, 100)
Reg_s_weighted = assign_weights_randomly(Reg_s)

Reg_m = nx.random_regular_graph(50, 500)
Reg_m_weighted = assign_weights_randomly(Reg_m)

Reg_l = nx.random_regular_graph(100, 1000)
Reg_l_weighted = assign_weights_randomly(Reg_l)

# 3. ER graphs

ER_s = nx.erdos_renyi_graph(100, 0.1)
ER_s_weighted = assign_weights_randomly(ER_s)

ER_m = nx.erdos_renyi_graph(500, 0.1)
ER_m_weighted = assign_weights_randomly(ER_m)

ER_l = nx.erdos_renyi_graph(1000, 0.1)
ER_l_weighted = assign_weights_randomly(ER_l)

# 4. Lobster

Lobster_s = nx.random_lobster(100, 0.1, 0.1)
Lobster_s_weighted = assign_weights_randomly(Lobster_s)

Lobster_m = nx.random_lobster(500, 0.1, 0.1)
Lobster_m_weighted = assign_weights_randomly(Lobster_m)

Lobster_l = nx.random_lobster(1000, 0.1, 0.1)
Lobster_l_weighted = assign_weights_randomly(Lobster_l)

# 5. Watts-Strogatz

WS_s = nx.watts_strogatz_graph(100, 5, 0.1)
WS_s_weighted = assign_weights_randomly(WS_s)

WS_m = nx.watts_strogatz_graph(500, 5, 0.1)
WS_m_weighted = assign_weights_randomly(WS_s)

WS_l = nx.watts_strogatz_graph(1000, 5, 0.1)
WS_l_weighted = assign_weights_randomly(WS_l)

# 6. SUSC

SUSC_s = susc_generator(100, 3, 0.1)
SUSC_s_weighted = assign_weights_randomly(SUSC_s)

SUSC_m = susc_generator(500, 3, 0.1)
SUSC_m_weighted = assign_weights_randomly(SUSC_m)

SUSC_l = susc_generator(1000, 3, 0.1)
SUSC_l_weighted = assign_weights_randomly(SUSC_l)

# 7. Cycle
C_s = nx.cycle_graph(100)
C_s_weighted = assign_weights_randomly(C_s)

C_m = nx.cycle_graph(500)
C_m_weighted = assign_weights_randomly(C_m)

C_l = nx.cycle_graph(1000)
C_l_weighted = assign_weights_randomly(C_l)

# 8. Grid
G_s = nx.grid_2d_graph(10, 10)
G_s_weighted = assign_weights_randomly(G_s)

G_m = nx.grid_2d_graph(20, 20)
G_m_weighted = assign_weights_randomly(G_m)

G_l = nx.grid_2d_graph(30, 30)
G_l_weighted = assign_weights_randomly(G_l)

# 9. Ladder
L_s = nx.ladder_graph(50)
L_s_weighted = assign_weights_randomly(L_s)

L_m = nx.ladder_graph(250)
L_m_weighted = assign_weights_randomly(L_m)

L_l = nx.ladder_graph(500)
L_l_weighted = assign_weights_randomly(L_l)

# 10. Hypergraph
H_s = nx.hypercube_graph(7)
H_s_weighted = assign_weights_randomly(H_s)

H_m = nx.hypercube_graph(9)
H_m_weighted = assign_weights_randomly(H_m)

H_l = nx.hypercube_graph(10)
H_l_weighted = assign_weights_randomly(H_l)

small_graphs = [
    ("Small BA", BA_s_weighted), 
    ("Small Reg", Reg_s_weighted), 
    ("Small ER", ER_s_weighted), 
    ("Small Lobster", Lobster_s_weighted), 
    ("Small WS", WS_s_weighted), 
    ("Small SUSC", SUSC_s_weighted), 
    ("Small Cycle", C_s_weighted), 
    ("Small Grid", G_s_weighted), 
    ("Small Ladder", L_s_weighted), 
    ("Small Hyper", H_s_weighted)
]

medium_graphs = [
    ("Medium BA", BA_m_weighted), 
    ("Medium Reg", Reg_m_weighted), 
    ("Medium ER", ER_m_weighted), 
    ("Medium Lobster", Lobster_m_weighted), 
    ("Medium WS", WS_m_weighted), 
    ("Medium SUSC", SUSC_m_weighted), 
    ("Medium Cycle", C_m_weighted), 
    ("Medium Grid", G_m_weighted), 
    ("Medium Ladder", L_m_weighted), 
    ("Medium Hyper", H_m_weighted)
]

large_graphs = [
    ("Large BA", BA_l_weighted), 
    ("Large Reg", Reg_l_weighted), 
    ("Large ER", ER_l_weighted), 
    ("Large Lobster", Lobster_l_weighted), 
    ("Large WS", WS_l_weighted), 
    ("Large SUSC", SUSC_l_weighted), 
    ("Large Cycle", C_l_weighted), 
    ("Large Grid", G_l_weighted), 
    ("Large Ladder", L_l_weighted), 
    ("Large Hyper", H_l_weighted)
]

all_graphs = small_graphs + medium_graphs + large_graphs

graphs_by_type = [(small_graphs[i], medium_graphs[i], large_graphs[i]) for i in range(len(small_graphs))]

In [ ]:
def lightest_node(gcc, G):
  return min(gcc, key=lambda v: G.nodes[v]['node weight'])

def weighted_degree_centrality(gcc, G):
  # Page 7, https://computationalsocialnetworks.springeropen.com/articles/10.1186/s40649-020-00081-w
  degrees = nx.degree_centrality(G.subgraph(gcc))
  weighted_degrees = {}
  for u in gcc:
    weighted_degree = 0
    for v in G.neighbors(u):
      weighted_degree += G.nodes[v]['node weight']
    weighted_degrees[u] = weighted_degree
  return max(gcc, key=lambda v: weighted_degrees[v])

def betweenness_centrality_cd(gcc, G):
  centrality_dict = nx.betweenness_centrality(G.subgraph(gcc), normalized=True)
  return max(gcc, key=lambda v: centrality_dict[v] / G.nodes[v]['node weight'])

def approximate_betweenness_centrality_cd(gcc, G):
  centrality_dict = nx.betweenness_centrality(G.subgraph(gcc), k=max(1, int(np.log(len(gcc)))))
  return max(gcc, key=lambda v: centrality_dict[v] / G.nodes[v]['node weight'])


def betweenness_centrality_wsp(gcc, G):
  # https://www.hindawi.com/journals/complexity/2021/1677445/
  # We find betweenness centrality using weighted shortest paths.
  # The higher the weight of a shortest path, the heavier the nodes are on that path. Thus, disconnecting that path is more better.
  # (Is this true? TODO: Figure out what networkx is doing here: https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.betweenness_centrality.html)
  centrality_dict = nx.betweenness_centrality(G.subgraph(gcc), weight ='edge weight')
  return max(gcc, key=lambda v: centrality_dict[v])


def approximate_betweenness_centrality_wsp(gcc, G):
  centrality_dict = nx.betweenness_centrality(G.subgraph(gcc), weight='edge weight', k=max(1, int(np.log(len(gcc)))))
  return max(gcc, key=lambda v: centrality_dict[v])



def eigenvector_centrality_cd(gcc, G):
  # Page 7, https://computationalsocialnetworks.springeropen.com/articles/10.1186/s40649-020-00081-w
  # "Eigenvector centrality is a measure where it is still open how to include the efect of node weights."
  try:
    centrality_dict = nx.eigenvector_centrality_numpy(G.subgraph(gcc))
    return max(gcc, key=lambda v: centrality_dict[v] / G.nodes[v]['node weight'])
  except:
    return lightest_node(gcc, G)




def power_iteration(G, nodelist, max_iter=100):
    A = nx.to_numpy_array(G, nodelist=nodelist, weight='node weight')
    v = np.random.rand(A.shape[1])
    for _ in range(max_iter):
        v = np.dot(A, v)
        try:
          v = v / np.linalg.norm(v)
        except:
          print(_, v)
          x = 5 / 0 
    return v

def eigenvector_centrality_weighted_adj(gcc, G):
  gcc = list(gcc)
  v = power_iteration(G, gcc)
  return gcc[np.argmax(v)]


def eigenvector_centrality_combined(gcc, G):
  gcc = list(gcc)
  v = power_iteration(G, gcc)
  cd_v = [u / G.nodes[gcc[i]]['node weight'] for (i, u) in enumerate(v)]
  return max(gcc, key=lambda v: cd_v[gcc.index(v)])


def simplify_and_invert_laplacian(L):
  L = np.delete(L, 0, axis=0)
  L_tilde = np.delete(L, 0, axis=1)
  L_inv = np.linalg.inv(L_tilde)
  L_with_zeroes_on_top = np.vstack((np.zeros(L_inv.shape[1]), L_inv))
  zeroes_column = np.zeros((L_with_zeroes_on_top.shape[0],1))
  C = np.hstack((zeroes_column, L_with_zeroes_on_top))
  return C


def approx_current_flow_betweenness(gcc, G, using_cd=False, using_weighted_edges=False, using_weighted_supply=False):
  G = G.subgraph(gcc)
  gcc = sorted(list(gcc))
  n = G.number_of_nodes()

  vertices_ordered = {v: i for i, v in enumerate(G.nodes)}
  centrality = {v: 0 for v in G.nodes}

  C = simplify_and_invert_laplacian(nx.laplacian_matrix(G).toarray())

  # k=logn pivots are used for approximation
  for _ in range(int(np.log(n))):
    # Source and sink, chosen uniformly at random
    s, t = random.sample(gcc, k=2)

    # Supply vector b
    b = np.zeros(n)
    b[vertices_ordered[s]] = 1
    b[vertices_ordered[t]] = -1
    if using_weighted_supply: 
      b = b * (G.nodes[s]['node weight'] + G.nodes[t]['node weight'])

    # Potential difference vector p
    p = C @ b

    for v in G.nodes:
      for e in G.edges(v):
        u = e[0] if e[0] != v else e[1]
        direction_of_current = 1 if vertices_ordered[v] < vertices_ordered[u] else -1
        potential_difference = abs(p[vertices_ordered[v]] - p[vertices_ordered[u]])
        centrality[v] += direction_of_current * (1 / G.edges[e]['edge weight'] if using_weighted_edges else 1)

  best_v = max(gcc, key=lambda v: centrality[v] / (G.nodes[v]['node weight'] if using_cd else 1))
  return best_v 



cfbc_cd = lambda gcc, G: approx_current_flow_betweenness(gcc, G, using_cd=True)
cfbc_wedge = lambda gcc, G: approx_current_flow_betweenness(gcc, G, using_weighted_edges=True)
cfbc_wsup = lambda gcc, G: approx_current_flow_betweenness(gcc, G, using_weighted_supply=True)
cfbc_combined = lambda gcc, G: approx_current_flow_betweenness(gcc, G, using_cd=True, using_weighted_edges=True, using_weighted_supply=True)

In [ ]:
heuristics = [
  ## Extra-greedy heuristics
  ("lightest_node", lightest_node),
  ("weighted_degree_centrality", weighted_degree_centrality),
  
  ## Betweenness heuristics
  ("betweenness_centrality_cd", betweenness_centrality_cd),
  ("approximate_betweenness_centrality_cd", approximate_betweenness_centrality_cd),
  ("betweenness_centrality_wsp", betweenness_centrality_wsp),
  ("approximate_betweenness_centrality_wsp", approximate_betweenness_centrality_wsp),
  
  ## Eigenvector centralities
  ("eigenvector_centrality_cd", eigenvector_centrality_cd),
  ("eigenvector_centrality_weighted_adj", eigenvector_centrality_weighted_adj),
  ("eigenvector_centrality_combined", eigenvector_centrality_combined),

  ## Current-flow centralities
  ("cfbc_cd", cfbc_cd),
  ("cfbc_wedge", cfbc_wedge),
  ("cfbc_wsup", cfbc_wsup),
  ("cfbc_combined", cfbc_combined)
]

# Tests by size (DO NOT RUN)

In [ ]:
### Testing Small Graphs ###
results_small = test_graphs(small_graphs, "small", heuristics, timeout=180, c=0.2)

--- Running tests for small graphs ---

Graph: Small BA
n: 100 m: 900

Testing lightest_node
Removed 80 nodes from G.
Cost: 37.97637577752967
Time elapsed: 0.053s
Robustness score: 60.7375

Testing weighted_degree_centrality
Removed 59 nodes from G.
Cost: 31.40228247897161
Time elapsed: 0.063s
Robustness score: 69.61016949152543

~~~~~~~~~~~~~~~~~~~~~~~~~~~

Graph: Small Reg
n: 100 m: 500

Testing lightest_node
Removed 78 nodes from G.
Cost: 30.08121610189149
Time elapsed: 0.017s
Robustness score: 61.57692307692308

Testing weighted_degree_centrality
Removed 66 nodes from G.
Cost: 28.204502781712048
Time elapsed: 0.063s
Robustness score: 66.87878787878788

~~~~~~~~~~~~~~~~~~~~~~~~~~~

Graph: Small ER
n: 100 m: 501

Testing lightest_node
Removed 74 nodes from G.
Cost: 25.39763675189061
Time elapsed: 0.014s
Robustness score: 63.33783783783784

Testing weighted_degree_centrality
Removed 59 nodes from G.
Cost: 22.46475197461381
Time elapsed: 0.286s
Robustness score: 69.08474576271186

~~~~

In [ ]:
### Testing Medium Graphs ###
results_medium = test_graphs(medium_graphs, "medium", heuristics, timeout=180, c=0.2)

--- Running tests for medium graphs ---

Graph: Medium BA
n: 500 m: 22500

Testing lightest_node
Removed 400 nodes from G.
Cost: 148.7668100962576
Time elapsed: 0.757s
Robustness score: 300.75

Testing weighted_degree_centrality
Removed 398 nodes from G.
Cost: 177.00133219081778
Time elapsed: 5.12s
Robustness score: 301.73366834170855

~~~~~~~~~~~~~~~~~~~~~~~~~~~

Graph: Medium Reg
n: 500 m: 12500

Testing lightest_node
Removed 400 nodes from G.
Cost: 166.2700065970698
Time elapsed: 0.319s
Robustness score: 300.75

Testing weighted_degree_centrality
Removed 399 nodes from G.
Cost: 184.9002227451193
Time elapsed: 3.996s
Robustness score: 301.23308270676694

~~~~~~~~~~~~~~~~~~~~~~~~~~~

Graph: Medium ER
n: 500 m: 12347

Testing lightest_node
Removed 400 nodes from G.
Cost: 150.3448340011436
Time elapsed: 0.342s
Robustness score: 300.75

Testing weighted_degree_centrality


KeyboardInterrupt: ignored

In [ ]:
### Testing Large Graphs ###
results_large = test_graphs(large_graphs, "large", heuristics, timeout=180, c=0.2)

# Tests by graph type

In [ ]:
### Testing Barabási-Albert graphs ###
# Alex
ba_graphs = graphs_by_type[0]
results_ba = test_graphs(ba_graphs, "Barabási-Albert", heuristics, timeout=300, c=3)

--- Running tests for Barabási-Albert graphs ---

Graph: Small BA
n: 100 m: 900

Testing lightest_node
Removed 88 nodes from G.
Cost: 3855.705605006826
Removed 88 nodes from G.
Robustness score: 1552.7545301269163
Time elapsed: 0.023s

Testing weighted_degree_centrality
Removed 66 nodes from G.
Cost: 2984.166314013016
Removed 66 nodes from G.
Robustness score: 1830.8481450608288
Time elapsed: 0.064s

Testing betweenness_centrality_cd
Removed 72 nodes from G.
Cost: 3183.3062890115857
Removed 72 nodes from G.
Robustness score: 1544.3809618806772
Time elapsed: 3.157s

Testing approximate_betweenness_centrality_cd
Removed 73 nodes from G.
Cost: 3133.8921161878043
Removed 73 nodes from G.
Robustness score: 1574.4911493248667
Time elapsed: 0.197s

Testing betweenness_centrality_wsp
Removed 67 nodes from G.
Cost: 3200.871661787244
Removed 67 nodes from G.
Robustness score: 1840.4094800022308
Time elapsed: 7.822s

Testing approximate_betweenness_centrality_wsp
Removed 67 nodes from G.
Cost: 31

In [ ]:
### Testing Regular graphs ###
# Alex
reg_graphs = graphs_by_type[1]
results_reg = test_graphs(reg_graphs, "Regular", heuristics, timeout=300, c=3)

In [ ]:
### Testing Erdős-Rényi graphs ###
# Alex
er_graphs = graphs_by_type[2]
results_er = test_graphs(er_graphs, "Erdős-Rényi", heuristics, timeout=300, c=3)

In [ ]:
### Testing Lobster graphs ###
# Alex
lobster_graphs = graphs_by_type[3]
results_lobster = test_graphs(lobster_graphs, "Lobster", heuristics, timeout=300, c=3)

In [ ]:
### Testing Watts-Strogatz graphs ###
# Rishi
wa_graphs = graphs_by_type[4]
results_wa = test_graphs(wa_graphs, "Watts-Strogatz", heuristics, timeout=300, c=3)

In [ ]:
### Testing Sea-Urchin Sistine-Chapel graphs ###
# Rishi
susc_graphs = graphs_by_type[5]
results_susc = test_graphs(susc_graphs, "Sea-Urchin Sistine-Chapel", heuristics, timeout=300, c=3)

In [ ]:
### Testing Cycle graphs ###
# Rishi
cycle_graphs = graphs_by_type[6]
results_cycle = test_graphs(cycle_graphs, "Cycle", heuristics, timeout=300, c=3)

In [ ]:
### Testing Grid graphs ###
# Anthony
grid_graphs = graphs_by_type[7]
results_grid = test_graphs(grid_graphs, "Grid", heuristics, timeout=300, c=3)

In [ ]:
### Testing Ladder graphs ###
# Anthony
ladder_graphs = graphs_by_type[8]
results_ladder = test_graphs(ladder_graphs, "Ladder", heuristics, timeout=300, c=3)

--- Running tests for Ladder graphs ---

Graph: Small Ladder
n: 100 m: 148

Testing laplacian_centrality
Removed 51 nodes from G.
Cost: 2710.6792431133595
Removed 51 nodes from G.
Robustness score: 1003.346736950062
Time elapsed: 2.648s

~~~~~~~~~~~~~~~~~~~~~~~~~~~

Graph: Medium Ladder
n: 500 m: 748

Testing laplacian_centrality


In [ ]:
### Testing Hyper graphs ###
# Anthony
hyper_graphs = graphs_by_type[9]
results_hyper = test_graphs(hyper_graphs, "Hyper", heuristics, timeout=300, c=3)